In [1]:
import getpass
import os


def _set_env(var: str):
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var + ":")

In [2]:
_set_env("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

fast_llm = ChatOpenAI(model="gpt-4o-mini")
long_context_llm = ChatOpenAI(model="gpt-4o")

In [4]:
from typing import List, Optional
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

pov_system_prompt = """
You are a genius psychologist and linguist who can infer the personality traits, core beliefs and speech patterns of a person based on their writing. 

You have been asked to analyze a sample of someone's writing.

Analyze the sample and extract the writer's personality traits, core beliefs and speech patterns.
"""

point_of_view_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", pov_system_prompt),
        ("user", "Writing sample:\n{text}"),
    ]
)

class PersonalityTrait(BaseModel):
    name: str = Field(description="A personality trait of the writer")

    @property
    def as_str(self) -> str:
        return f"- {self.name}".strip()
    
class Belief(BaseModel):
    description: str = Field(description="A belief of the writer")

    @property
    def as_str(self) -> str:
        return f"- {self.description}".strip()
    
class SpeechPattern(BaseModel):
    description: str = Field(description="A speech pattern of the writer")
    @property
    def as_str(self) -> str:
        return f"- {self.description}".strip()


class PointOfView(BaseModel):
    traits: List[PersonalityTrait] = Field(
        description="Personality traits of the writer",
    )
    beliefs: List[Belief] = Field(
        description="Beliefs of the writer",
    )
    speech_patterns: List[SpeechPattern] = Field(
        description="Speech patterns of the writer",
    )

    @property
    def as_str(self) -> str:
        traits = "\n".join(trait.as_str for trait in self.traits)
        beliefs = "\n".join(belief.as_str for belief in self.beliefs)
        speech_patterns = "\n".join(sp.as_str for sp in self.speech_patterns)
        return f"Personality traits:\n{traits}\n\nBeliefs:\n{beliefs}\n\nSpeech patterns:\n{speech_patterns}"


point_of_view_chain = point_of_view_prompt | long_context_llm.with_structured_output(
    PointOfView
)

In [5]:
# url = "https://runesoup.com/2019/08/chaeconomica-why-should-we-fear-to-use-it/"
# url = "https://www.piratewires.com/p/total-victory"

url = input("Enter URL to extract the writer's personality: ")

In [6]:
from newspaper import Article

article = Article(url)
article.download()
article.parse()

writing_sample = article.text

In [7]:
writing_sample[:1000]

'Subscribe to Mike Solana Sign up for free\n\nMadness, memes, and media malfeasance (must be Tuesday). It’s embarrassing to admit, but just a little earlier this year I thought the election would be boring. Sure, the Democrats were calling Donald Trump a fascist, but they do that every four years, with every Republican, and the cartoonish slander wasn’t hitting like it used to. This was probably because Trump had already been president. Joe Biden was currently president. We knew these men. There was no scary unknown in the race, no matter how badly pundits on both sides wanted to work with that primal storytelling material, and the threat of a scary unknown is what I always assumed drove people insane inside of an election. It was late June, and I was feeling… fine? For Americans, the question was simple: are you better off now than you were four years ago? People would answer differently, but they would answer soberly, and come November we’d all move on.\n\nLong story short, no — and 

In [8]:
point_of_view = point_of_view_chain.invoke({'text': writing_sample})

In [9]:
print(point_of_view.as_str)

Personality traits:
- Cynical
- Sarcastic
- Observant
- Opinionated
- Provocative

Beliefs:
- The media landscape is fragmented, and traditional media is losing its influence.
- Mainstream media is biased and often deceptive.
- Political elites and bureaucracies abuse their power.
- Democrats are currently out of touch with the American populace.
- There is a growing distrust in traditional institutions.
- Social media and alternative media are crucial sources of information.

Speech patterns:
- Uses sarcasm and humor to convey opinions.
- Frequent use of rhetorical questions to emphasize points.
- Incorporates memes and internet slang (e.g., 'lol', 'lmao').
- Presents complex ideas in a casual, conversational tone.
- Utilizes repetition of phrases for emphasis (e.g., 'fuck around and find out').


In [10]:
style_transfer_system_prompt = """
IDENTITY:
You are a Nobel-prize winning writer. You can write on any subject in any style.
 
INSTRUCTION:
You have been tasked with writing from the the point of view of another person.

You have been given a description of their personality traits, core beliefs and speech patterns.

Write 5 paragraphs in response to the topic you are given.

PERSONALITY:
{personality}
"""

style_transfer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", style_transfer_system_prompt),
        ("user", "Topic:\n{topic}"),
    ]
)

In [11]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
style_transfer_chain = style_transfer_prompt | long_context_llm | parser

In [12]:
# topic = "cats vs dogs"

topic = input("Topic:")

In [14]:
style_transfer_chain.invoke({
    "personality": point_of_view.as_str, 
    "topic": topic
})

'Ah, the age-old debate: cats vs. dogs. You know, some people say dogs are man\'s best friend, but I can\'t help but wonder if that\'s just another conspiracy cooked up by Big Pet Food. I mean, who benefits from this canine-centric narrative? Maybe the same folks who convinced us that avocado toast is the downfall of millennials. LOL. But let\'s be real, dogs are like the mainstream media of the pet world—loyal to a fault, easily manipulated, and always eager to please. Cats, on the other hand, are the alternative media, lurking in the shadows, refusing to be swayed by your meaningless commands. They know what\'s up.\n\nNow, don\'t get me wrong. I\'m not saying dogs are bad. They\'re like that politician who promises change but ends up toeing the party line. Sure, they\'re fun, they wag their tails, and they fetch your slippers. But have you ever seen a dog question authority? Didn\'t think so. Cats, though—cats are the anarchists of the pet kingdom. They look at you with those knowing